In [ ]:
# Load environment variables from parent directory and set up auto-reload
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

## Context Isolation: Sub-agents

#TODO(Geoff): Can add more here. 

* Agent context can grow quickly. 
* Agents face several long context-related problems.
* A primary problem is context clash or confusion. 
* Context with mixed objectives can lead to suboptimal performance.
* [Context isolation](https://blog.langchain.com/context-engineering-for-agents/) is a good way to address these problems.
* We can delegate tasks to [specialized sub-agents](https://www.anthropic.com/engineering/multi-agent-research-system).
* Each sub-agent has its own context window.
* This prevents context clashes, confusion, poisoning, and dilution.

### Sub-agent delegation

* The primary insight is that we can create sub-agents with different tool sets.
* We can invoke them with a tool call, `task(description, subagent_type)`.
* To this too we pass the parameters needed to create the sub-agent.
* The work done by the sub-agent is returned as a `ToolMessage` to the parent agent.

#TODO(Geoff): We will need to explain all of this. A bit dense.

In [ ]:
from typing import Annotated, NotRequired, TypedDict

from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.types import Command

from deep_agents_from_scratch.prompts import (
    TASK_DESCRIPTION_PREFIX,
    TASK_DESCRIPTION_SUFFIX,
)
from deep_agents_from_scratch.state import DeepAgentState


class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""
    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]

def _create_task_tool(tools, instructions, subagents: list[SubAgent], model, state_schema):
    agents = {
        "general-purpose": create_react_agent(model, prompt=instructions, tools=tools)
    }
    tools_by_name = {}
    for tool_ in tools:
        if not isinstance(tool_, BaseTool):
            tool_ = tool(tool_)
        tools_by_name[tool_.name] = tool_
    for _agent in subagents:
        if "tools" in _agent:
            _tools = [tools_by_name[t] for t in _agent["tools"]]
        else:
            _tools = tools
        agents[_agent["name"]] = create_react_agent(
            model, prompt=_agent["prompt"], tools=_tools, state_schema=state_schema
        )

    other_agents_string = [
        f"- {_agent['name']}: {_agent['description']}" for _agent in subagents
    ]

    @tool(
        description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string)
        + TASK_DESCRIPTION_SUFFIX
    )
    def task(
        description: str,
        subagent_type: str,
        state: Annotated[DeepAgentState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        if subagent_type not in agents:
            return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"
        sub_agent = agents[subagent_type]
        state["messages"] = [{"role": "user", "content": description}]
        result = sub_agent.invoke(state)
        return Command(
            update={
                "files": result.get("files", {}),
                "messages": [
                    ToolMessage(
                        result["messages"][-1].content, tool_call_id=tool_call_id
                    )
                ],
            }
        )

    return task